In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
data = np.load('data_linprog.npz')
X = data['X']
y = data['y']
n, k = X.shape
print(f'{n=}, {k=}')

n=10, k=20


Since $n < k$, there exist many vectors $a \in \mathbb{R}^k $ such that $y_i = a\cdot x_i$ (i.e. $Xa = y$).
Let's find the vector $a$ siths smallest $L_1$ norm. This is a linear programming problem:

Find $\min_{a \in \mathbb{R}^k} \| a \|$ such that $ Xa = y $.

This is indeed a linear program, but we should rewrite it in order to pass to `scipy.optimize.linprog`:

Find $ \min_t 1^Tt $ such that $ Xa = y $ and $ a_i \leqslant t_i,\ a_i \geqslant -t_i $ for all $i$.

And in even more verbose form:

$$ \min_{a,t} \begin{bmatrix} 0 & 1 \end{bmatrix} \cdot \begin{bmatrix} a \\ t \end{bmatrix} $$
such that
$$ \begin{bmatrix} X & 0 \end{bmatrix} \cdot \begin{bmatrix} a \\ t \end{bmatrix} = y, $$
$$ \begin{bmatrix} \hat{I} & -\hat{I} \\ -\hat{I} & -\hat{I} \end{bmatrix} \cdot \begin{bmatrix} a \\ t \end{bmatrix} \leqslant 0, $$
$$ \begin{bmatrix} -\infty \\ 0 \end{bmatrix} \leqslant
    \begin{bmatrix} a \\ t \end{bmatrix} \leqslant
    \begin{bmatrix} +\infty \\ +\infty \end{bmatrix}. $$

In terms of scipy.optimize.linprog:

$$ A_{ub} = \begin{bmatrix} \hat{I} & -\hat{I} \\ -\hat{I} & -\hat{I} \end{bmatrix}_{2k \times 2k}\ b_{ub} = 0_{2k} $$
$$ A_{eq} = \begin{bmatrix} X & 0 \end{bmatrix}_{n \times 2k}\ b_{eq} = y_n $$
$$ l = \begin{bmatrix} -\infty \\ 0 \end{bmatrix}_{2k}\  u = \begin{bmatrix} +\infty \\ +\infty \end{bmatrix}_{2k}$$

In [22]:
c = np.block([np.zeros(k), np.ones(k)])
A_ub = np.block([[np.identity(k), -np.identity(k)], [-np.identity(k), -np.identity(k)]])
b_ub = np.zeros(2 * k)
A_eq = np.block([X, np.zeros([n, k])])
l = np.block([np.full(k, -np.inf), np.zeros(k)])
u = np.full(2 * k, np.inf)
result = linprog(c, A_ub, b_ub, A_eq, y, bounds=np.block([[l], [u]]).transpose(), method='highs')
print(result)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 9.634439501271336
              x: [-1.602e+00 -4.835e-13 ...  1.163e+00  2.855e+00]
            nit: 34
          lower:  residual: [       inf        inf ...  1.163e+00
                              2.855e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 3.152e-01  5.945e-01 -4.395e-01  2.630e-02
                              4.893e-01  5.334e-01  7.701

In [28]:
print(result.x[:20])
print(f'Non-zero elements: {np.count_nonzero(result.x[:20])}')
print(f'Significantly non-zero elements: {sum(list(map(lambda x: 0 if abs(x) < 1e-12 else 1, result.x[:20])))}')

[-1.60245750e+00 -4.83459789e-13 -2.48224304e-13  9.10950467e-02
 -0.00000000e+00 -2.82416835e-01 -0.00000000e+00  4.20867369e-13
  1.68359791e+00 -4.36602158e-13 -0.00000000e+00 -0.00000000e+00
  1.60475248e-01  1.25539647e-01 -5.82053763e-14  4.45693938e-13
 -1.49968049e-01 -1.52127518e+00 -1.16251561e+00  2.85509849e+00]
Non-zero elements: 16
Significantly non-zero elements: 10
